In [36]:
import spacy
from spacy_readability import Readability
import json
import tabulate
dictionary = {}
dictionary_lemma = {}
dictionary_adj = {}
dictionary_noun = {}
dictionary_verb = {}
nlp = spacy.load("de_core_news_sm")

def createLibrary(data):
    for d in data:
        tp = 0 
        tn = 0
        fp = 0
        fn = 0
        if d['label'] and d['prediction']:
            tp = 1
        elif not d['label'] and not d['prediction']:
            tn = 1
        elif not d['label'] and d['prediction']:
            fp = 1
        elif d['label'] and not d['prediction']:
            fn = 1
        
        for t in d['tokens']:
            if t.text in dictionary:
                dictionary[t.text] =  {
                    "count": dictionary[t.text]['count'] + 1,
                    "tp": dictionary[t.text]['tp'] + tp,
                    "tn": dictionary[t.text]['tn'] + tn,
                    "fp": dictionary[t.text]['fp'] + fp,
                    "fn": dictionary[t.text]['fn'] + fn
                }
            else:
                dictionary[t.text] =  {"count": 1, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
            if t.lemma_ in dictionary_lemma:
                dictionary_lemma[t.lemma_] =  {
                    "count": dictionary_lemma[t.lemma_]['count'] + 1,
                    "tp": dictionary_lemma[t.lemma_]['tp'] + tp,
                    "tn": dictionary_lemma[t.lemma_]['tn'] + tn,
                    "fp": dictionary_lemma[t.lemma_]['fp'] + fp,
                    "fn": dictionary_lemma[t.lemma_]['fn'] + fn
                }
            else:
                dictionary_lemma[t.lemma_] =  {"count": 1, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
            if t.pos_ == "ADJ":
                if t.text in dictionary_adj:
                    dictionary_adj[t.text] =  {
                    "count": dictionary_adj[t.text]['count'] + 1,
                    "tp": dictionary_adj[t.text]['tp'] + tp,
                    "tn": dictionary_adj[t.text]['tn'] + tn,
                    "fp": dictionary_adj[t.text]['fp'] + fp,
                    "fn": dictionary_adj[t.text]['fn'] + fn
                }
                else:
                    dictionary_adj[t.text] =  {"count": 1, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
            if t.pos_ == "NOUN":
                if t.text in dictionary_noun:
                    dictionary_noun[t.text] =  {
                    "count": dictionary_noun[t.text]['count'] + 1,
                    "tp": dictionary_noun[t.text]['tp'] + tp,
                    "tn": dictionary_noun[t.text]['tn'] + tn,
                    "fp": dictionary_noun[t.text]['fp'] + fp,
                    "fn": dictionary_noun[t.text]['fn'] + fn
                }
                else:
                    dictionary_noun[t.text] =  {"count": 1, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
            if t.pos_ == "VERB":
                if t.text in dictionary_verb:
                    dictionary_verb[t.text] =  {
                    "count": dictionary_verb[t.text]['count'] + 1,
                    "tp": dictionary_verb[t.text]['tp'] + tp,
                    "tn": dictionary_verb[t.text]['tn'] + tn,
                    "fp": dictionary_verb[t.text]['fp'] + fp,
                    "fn": dictionary_verb[t.text]['fn'] + fn
                }
                else:
                    dictionary_verb[t.text] =  {"count": 1, "tp": 0, "tn": 0, "fp": 0, "fn": 0}
def printFirst100():
    s = sorted(dictionary, key=dictionary.get, reverse=True)
    def lengthFilter(e):
        return len(e)>3 and (len(e) > 5 or e[0].isupper())
    s = list(filter(lengthFilter, s))
    for i in range(100):
        print(s[i])

with open('predictions.json') as json_file:
    data = json.load(json_file)
#     print(data)
    for comment in data:
        doc = nlp(comment['text'])
        comment['tokens'] = doc
    
    createLibrary(data)
#     printFirst100()

In [30]:
for t in nlp('Du bist Arschloch.!,'):
    print(t, t.pos_)

Du PRON
bist AUX
Arschloch NOUN
. PUNCT
! PUNCT
, PUNCT


In [52]:
from IPython.display import HTML, display

def defaultFilter(el):
    return True

def calcMetrics(f = defaultFilter):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for el in filter(f, data):
        if el['label'] and el['prediction']:
            tp += 1
        if not el['label'] and not el['prediction']:
            tn += 1
        if el['label'] and not el['prediction']:
            fp += 1
        if not el['label'] and el['prediction']:
            fn += 1
    accuracy = 0
    if (tp + fp + fn + tn) != 0:
        accuracy = (tp + tn) / (tp + fp + fn + tn)
    
    precision = 0
    if (tp + fp) != 0:
        precision = tp / (tp + fp)
    
    recall = 0
    if (tp + fn) != 0:
        recall = tp / (tp + fn)
    
    f1Score = 0
    if (recall + precision) != 0:
        f1Score = (2 * (recall * precision)) / (recall + precision)
    return {
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "true": tp+tn,
        "false": fp+fn,
        "positiv": tp+fp,
        "negativ": tn+fn,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1Score": f1Score
    }
def showMetrics(metrics):
    display(HTML(tabulate.tabulate([
        ['positiv labels (%s)' % (metrics['tp']+metrics['fn']), 'negative labels (%s)' % (metrics['fp']+metrics['tn'])],
    ], tablefmt='html')))
    display(HTML(tabulate.tabulate([
        ['', 'true (%s)' % metrics['true'], 'false (%s)' % metrics['false']],
        ['positiv (%s)' % metrics['positiv'], metrics['tp'], metrics['fp']],
        ['negativ (%s)' % metrics['negativ'], metrics['tn'], metrics['fn']],
    ], tablefmt='html')))
    display(HTML(tabulate.tabulate([
        ['accuracy', 'precision', 'recall', 'f1Score'],
        [metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1Score']],
    ], tablefmt='html')))
def showData(f = defaultFilter):
    table = []
    table.append(['id', 'label', 'prediction', 'text', 'probability'])
    
    for el in filter(f, data):
        table.append([el['id'], el['label'], el['prediction'], el['text'], el['probability']])
        
    display(HTML(tabulate.tabulate(table, tablefmt='html')))

# showData()

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

def createPlt(name, filters, relative):
    names = []
    positions = range(len(filters))
    falseBar = [0] * len(filters)
    trueBar = [0] * len(filters)
    index = 0
    for f in filters:
        names.append(f[0])
        for element in filter(f[1], data):
            if element["label"] == element["prediction"]:
                trueBar[index] += 1
            else:
                falseBar[index] += 1
        index += 1
    if relative:
        index = 0
        for f in filters:
            if len(list(filter(f[1], data))) > 0:
                trueBar[index] = 100 * trueBar[index] / len(list(filter(f[1], data)))
                falseBar[index] = 100 * falseBar[index] / len(list(filter(f[1], data)))
            else:
                trueBar[index] = 0
                falseBar[index] = 0
            index += 1
            
    rc('font', weight='bold')
    barWidth = 0.5
    plt.bar(positions, trueBar, color='green', edgecolor='white', width=barWidth)
    plt.bar(positions, falseBar, bottom=trueBar, color='red', edgecolor='white', width=barWidth)
 
    plt.xticks(positions, names, fontweight='regular')
    plt.xlabel(name)
    plt.show()